Đây là script chạy huấn luyện và đánh giá mô hình gồm các bước:
- Clone repository từ GitHub
- Load dữ liệu đề bài của cuộc thi từ thư mục data/DATA
- Xử lý, tách dữ liệu thành hai nhóm Fresher và Senior
- Chạy tối ưu tham số cho mô hình
- Huấn luyện mô hình với tham số tối ưu
- Lưu mô hình đã huấn luyện cũng như file submission cuối cùng

**Lưu ý**:
- Script này **không thể** chạy local do thời gian chạy lớn và ngốn bộ nhớ. Chúng tôi đã đề xuất hai phương án thay thế kèm hướng dẫn để chạy là chạy trên [Google Colab](./HOW_TO_RUN_COLAB.md) và chạy trên [Kaggle](./HOW_TO_RUN_KAGGLE.md).
- Do đây là notebook chạy chung cho **12 trường hợp** của dự án trustee (3 hướng tiếp cận x 4 mô hình) nên để chạy hết sẽ rất tốn thời gian (phần lớn là do tối ưu hóa tham số). Thời gian chạy cụ thể cho từng trường có thể xem chi tiệt tại [THỜI GIAN CHẠY CHO TỪNG TRƯỜNG HỢP](../model/MODEL_HYPERPARAMETERS.md). Với những trường hợp có thời gian chạy quá lớn (>40 phút), chúng tôi khuyến khích nên chạy trên Kaggle do có cơ chế chạy offline.
- Có hai cell code cần lưu ý là cell 2 và cell 8:
    - Cell 2: code thay đổi tùy theo bạn chạy trên Google Colab hay Kaggle
    - Cell 8: hai biến MODEL_NAME và APPROACH_TYPE thay đổi tùy theo hướng tiếp cận và mô hình bạn chọn

In [ ]:
!git clone https://github.com/CryAndRRich/trustee.git

In [ ]:
# Nếu chạy trên Google Colab, uncomment 5 dòng dưới
# %cd /content/trustee
# TRUSTEE_PATH = "/content/trustee"
# !unzip "/content/data_for_scripts.zip" -d "/content/data_for_scripts"
# INPUT_ROOT = "/content/data_for_scripts/DATA"
# WORK_DIR = "/content/working"

# Nếu chạy trên Kaggle, uncomment 4 dòng dưới
# %cd /kaggle/working/trustee
# TRUSTEE_PATH = "/kaggle/working/trustee"
# INPUT_ROOT = "/kaggle/input/df2026-hd4k/DATA"
# WORK_DIR = "/kaggle/working"

In [ ]:
!pip install -r requirements.txt

In [3]:
import sys

if TRUSTEE_PATH not in sys.path:
    sys.path.append(TRUSTEE_PATH)

In [4]:
import numpy as np
import pandas as pd
import category_encoders as ce

In [5]:
from config import *
from preprocess import *
from utils import *
from model import *

In [ ]:
# Set up môi trường đảm bảo tính tái lập (hàm set_seed được định nghĩa trong utils/set_up.py)
RANDOM_SEED = 42
set_seed(RANDOM_SEED)

DEVICE = "cpu"

In [7]:
ACADEMIC_CSV = f"{INPUT_ROOT}/academic_records.csv"
ADMISSION_CSV = f"{INPUT_ROOT}/admission.csv"
TEST_CSV = f"{INPUT_ROOT}/test.csv"

SUBMISSION_CSV = f"{WORK_DIR}/submission.csv"

# MODEL_NAME là "Decision Tree", "Random Forest", "XGBoost" hoặc "LightGBM"
MODEL_NAME = "Decision Tree"

# APPROACH_TYPE là "Credits", "Gap" hoặc "Ratio"
APPROACH_TYPE = "Credits"
map_target = {
    "Credits": "TC_HOANTHANH",
    "Gap": "FAIL_CREDITS",
    "Ratio": "PASS_RATIO"
}

N_TRIALS = 300

In [8]:
# Load và xử lý dữ liệu (hàm get_data được định nghĩa trong preprocess/process_data.py)
academic_df, student_df, train_df_raw, val_df_raw, test_df_raw = get_data(
    ADMISSION_CSV, ACADEMIC_CSV, TEST_CSV
)

In [9]:
# Tạo đặc trưng (hàm get_data được định nghĩa trong preprocess/process_data.py)
train_final = get_features(train_df_raw, academic_df, student_df)
val_final = get_features(val_df_raw, academic_df, student_df)
test_final = get_features(test_df_raw, academic_df, student_df)

In [ ]:
# target sẽ là "TC_HOANTHANH", "FAIL_CREDITS" hoặc "PASS_RATIO" tùy theo APPROACH_TYPE
target = map_target[APPROACH_TYPE]
categorical_cols = ["PTXT", "TOHOP_XT"]

cbe = ce.CatBoostEncoder(
    cols=categorical_cols, 
    handle_missing="return_nan"
)

train_final[categorical_cols] = cbe.fit_transform(
    train_final[categorical_cols], 
    train_final[target]
)

val_final[categorical_cols] = cbe.transform(val_final[categorical_cols])
test_final[categorical_cols] = cbe.transform(test_final[categorical_cols])

print(f"Final Train shape: {train_final.shape}")
print(f"Final Val shape: {val_final.shape}")
print(f"Final Test shape: {test_final.shape}")

In [11]:
# Tách dữ liệu thành hai nhóm Fresher và Senior (hàm split_by_year được định nghĩa trong preprocess/process_data.py)
train_fresh, train_senior = split_by_year(train_final)
val_fresh, val_senior = split_by_year(val_final)
test_fresh, test_senior = split_by_year(test_final)

In [12]:
feats_senior = [
    "TC_DANGKY", "SEMESTER_INDEX", "SV_NAM_THU",
    
    "LAST_GPA", "LAST_FAIL", "LAST_PASS_RATIO",
    
    "R2_AVG_GPA", "R2_SUM_FAIL", "R2_PASS_RATE",
    "FAIL_TREND_R2", "GPA_TREND_R2",
    
    "R3_AVG_GPA", "R3_SUM_FAIL",
    "PRESSURE_VS_R2", "PRESSURE_VS_R3", "OVERLOAD_R3",
    
    "TOTAL_EARNED", "OVERLOAD_VS_MAX",
    "HIST_AVG_GPA", "HIST_MAX_PASSED", "HIST_MAX_GPA", "HIST_STD_GPA",    
]

feats_fresh = [
    "TC_DANGKY", "SEMESTER_INDEX", "PTXT", "TOHOP_XT",
    
    "DIEM_TRUNGTUYEN", "DIEM_CHUAN", 
    "SCORE_GAP", "ENTRY_RANK", "BENCHMARK_TIER",
    "Z_SCORE", "GAP_RATIO",

    "LAST_GPA", "LAST_FAIL", "LAST_PASS_RATIO",
    "PRESSURE_VS_R2" 
]

# Nếu target = "TC_HOANTHANH" thì không cần cộng vào meta_cols
meta_cols = ["MA_SO_SV", "HOC_KY", "TC_HOANTHANH"]
if target != "TC_HOANTHANH":
    meta_cols = meta_cols + [target]

In [13]:
# Lọc các cột thừa, chỉ giữ lại các cột cần thiết (hàm filter_cols được định nghĩa trong preprocess/process_data.py)
train_fresh = filter_cols(train_fresh, feats_fresh, meta_cols)
val_fresh = filter_cols(val_fresh, feats_fresh, meta_cols)
test_fresh = filter_cols(test_fresh, feats_fresh, meta_cols)

train_senior = filter_cols(train_senior, feats_senior, meta_cols)
val_senior = filter_cols(val_senior, feats_senior, meta_cols)
test_senior = filter_cols(test_senior, feats_senior, meta_cols)

full_train_fresh = pd.concat([train_fresh, val_fresh], axis=0, ignore_index=True)
full_train_senior = pd.concat([train_senior, val_senior], axis=0, ignore_index=True)

In [ ]:
print(f"Train Fresher: {train_fresh.shape} | Train Senior: {train_senior.shape}")
print(f"Val Fresher: {val_fresh.shape} | Val Senior: {val_senior.shape}")
print(f"Test Fresher: {test_fresh.shape} | Test Senior: {test_senior.shape}")

In [ ]:
# Tối ưu siêu tham số cho cả hai nhóm Fresher và Senior 
# (hàm optimize_params được định nghĩa trong model/hypertuning/__init__.py)
best_fresh_params, best_fresh_rmse = optimize_params(
    model_name=MODEL_NAME, 
    train_df=train_fresh,    
    val_df=val_fresh,
    feats=feats_fresh, 
    target_col=target,       
    n_trial=N_TRIALS, 
    model_type="Fresher", 
    approach_type=APPROACH_TYPE
)

best_senior_params, best_senior_rmse = optimize_params(
    model_name=MODEL_NAME,
    train_df=train_senior,   
    val_df=val_senior,
    feats=feats_senior, 
    target_col=target,
    n_trial=N_TRIALS, 
    model_type="Senior", 
    approach_type=APPROACH_TYPE
)

In [ ]:
print(f"Best Fresher RMSE: {best_fresh_rmse:.4f}")
print(f"Best Senior RMSE: {best_senior_rmse:.4f}")

print(f"Best Fresher Params: {best_fresh_params}")
print(f"Best Senior Params: {best_senior_params}")

In [ ]:
# Huấn luyện mô hình với siêu tham số tốt nhất và tạo dự đoán trên tập validation
# (hàm train_model được định nghĩa trong model/train/__init__.py)
best_iter_fresh, val_preds_fresh = train_model(
    model_name=MODEL_NAME, 
    params=best_fresh_params,
    train_df=train_fresh,    
    val_df=val_fresh,
    feats=feats_fresh, 
    target_cols=target,       
    model_type="Fresher", 
    approach_type=APPROACH_TYPE
)

best_iter_senior, val_preds_senior = train_model(
    model_name=MODEL_NAME, 
    params=best_senior_params,
    train_df=train_senior,    
    val_df=val_senior,
    feats=feats_senior, 
    target_cols=target,       
    model_type="Senior", 
    approach_type=APPROACH_TYPE
)

In [ ]:
val_targets = np.concatenate([val_fresh["TC_HOANTHANH"].values, val_senior["TC_HOANTHANH"].values])
val_preds = np.concatenate([val_preds_fresh, val_preds_senior])

# Tính toán các chỉ số đánh giá mô hình trên tập validation 
# (hàm evaluate_model_performance được định nghĩa trong utils/evaluate.py)
metrics = evaluate_model_performance(val_targets, val_preds)

In [ ]:
# Chạy dự đoán trên tập test với mô hình đã huấn luyện
# (hàm test_model được định nghĩa trong model/test/__init__.py)
test_preds_fresh = test_model(
    model_name=MODEL_NAME,
    params=best_fresh_params, 
    best_iter=best_iter_fresh, 
    full_train_df=full_train_fresh, 
    train_df=train_fresh, 
    test_df=test_fresh, 
    feats=feats_fresh, 
    target_col=target,
    save_dir=WORK_DIR,
    model_type="Fresher", 
    approach_type=APPROACH_TYPE
)

test_preds_senior = test_model(
    model_name=MODEL_NAME,
    params=best_senior_params, 
    best_iter=best_iter_senior, 
    full_train_df=full_train_senior, 
    train_df=train_senior, 
    test_df=test_senior, 
    feats=feats_senior, 
    target_col=target,
    save_dir=WORK_DIR,
    model_type="Senior", 
    approach_type=APPROACH_TYPE
)

In [ ]:
# Lưu kết quả dự đoán vào file submission.csv 
# (hàm save_predictions được định nghĩa trong utils/save_submission.py)
submission_df = save_predictions(
    test_fresh, test_senior,
    test_preds_fresh, test_preds_senior,
    test_df_raw,
    SUBMISSION_CSV
)

In [ ]:
print(submission_df)